# Single Machine Scheduling

This notebook includes <br>
- a problem generator: ()
- a disjunctive MIP model, 
- a time-indexed MIP model, 
- a rank-based MIP model, and 
- a linear-ordering MIP model 


In [130]:
import sys, os, datetime
import numpy as np
import string
import pandas as pd
import random
from gurobipy import *

In [131]:
####################################################################
def generate_SMS(N_, dmax, rmax, wmax):
    '''
    generate_SMS: generates an SMS problem with release dates
    the user should specify three parameters:
    N_: # of jobs, 
    rmax: maximum release date, 
    dmax: maximum duration
    range is from 0 to N-1 in program
    range is from 1 to N in the output file
    '''
    sms = []
    dmax_new = -1     # dmax_new: max duration of act, initialize as -1
    rmax_new = -1     # rmax_new: max release date of act, initialize as -1
    wmax_new = -1     # wmax_new: max weight of completion time, initialize as -1
    
    print('The maximum possible duration is: ', dmax)
    print('The maximum possible release date is: ', rmax)
    print('The maximum possible weight of completion time is: ', wmax)
    print('The number of jobs is: ', N_)
    print('The SMS is represented as below: ')
    
    dur = np.random.uniform(1, dmax+1, N_)
    rel = np.random.uniform(0, rmax+1, N_)
    weight = np.random.uniform(1, wmax+1, N_)
    
    for i in range(N_):
        sms.append((int(dur[i]), int(rel[i]), int(weight[i])))
    
    dmax_new = int(max(dur))
    dmin_new = int(min(dur))
    rmax_new = int(max(rel))
    rmin_new = int(min(rel))
    wmax_new = int(max(weight))
    wmin_new = int(min(weight))
    print('The real maximum duration is: ', dmax_new)
    print('The real minimum duration is: ', dmin_new)
    print('The real maximum release date is: ', rmax_new)
    print('The real minimum release date is: ', rmin_new)
    print('The real maximum weight of completion time is: ', wmax_new)
    print('The real minimum weight of completion time is: ', wmin_new)
    print('The SMS problem is: ', sms)
    
    return sms, dmax_new, dmin_new, rmax_new, rmin_new, wmax_new, wmin_new

In [132]:
###########################################################################
def outputSMS(sms, count, N_, dmax, dmin, rmax, rmin, wmax, wmin):
    '''
    Write a SMS instance into a .txt file
    '''
    filename = 'sms_rel_' + str(N_) + '_' + str(count) + '.txt'
    print(filename)
    open(filename,'w').close()
    f = open(filename,'a')
    f.write(str(N_)+' ')
    f.write(str(dmax)+' ')
    f.write(str(dmin)+' ')
    f.write(str(rmax)+' ')
    f.write(str(rmin)+' ')
    f.write(str(wmax)+' ')
    f.write(str(wmin)+' ')
    for i in range(len(sms)):
        f.write('\n')        
        f.write(str(sms[i][0])+' ')
        f.write(str(sms[i][1])+' ')
        f.write(str(sms[i][2]))

In [133]:
####################################################################
def readSMS(count, N_):
    '''
    Read a file in the folder, then give the QUBO formulation
    '''
    sms = []
    bound = []
    filename = 'sms_rel_' + str(N_) + '_' + str(count)+'.txt'
    f = open(filename, 'r')
    
    param = f.readline()
    list_param = param.split()
    N = int(list_param[0])
    dmax = int(list_param[1])
    dmin = int(list_param[2])
    rmax = int(list_param[3])
    rmin = int(list_param[4])
    wmax = int(list_param[5])
    wmin = int(list_param[6])
    
    duration = []
    release = []
    weight = []
    
    for i in range(N_):
        act = f.readline()
        list_act = act.split()
        duration.append(int(list_act[0]))
        release.append(int(list_act[1]))
        weight.append(int(list_act[2]))
        
    print("The number of activities: ", N_)
    print("The maximum duration: ", dmax)
    print("The minimum duration: ", dmin)
    print("The maximum release date: ", rmax)
    print("The minimum release date: ", rmin)
    print("The maximum weight of completion time: ", wmax)
    print("The minimum weight of completion time: ", wmin)
    print('The duration list: ', duration)
    print('The release date list: ', release)
    print('The weight list: ', weight)
    
    return N_, dmax, dmin, rmax, rmin, wmax, wmin, duration, release, weight

In [134]:
N_ = 80
dmax_init = 100
rmax_init = 2400
wmax_init = 1
count = 1
sms, dmax, dmin, rmax, rmin, wmax, wmin = generate_SMS(N_, dmax_init, rmax_init, wmax_init)
outputSMS(sms, count, N_, dmax, dmin, rmax, rmin, wmax, wmin)
N_, dmax, dmin, rmax, rmin, wmax, wmin, duration, release, weight = readSMS(count, N_)
# duration = [24,8,15,25,1,19,28,8,27,11]
# release = [23,22,16,23,27,14,7,14,9,9]
# duration = [7,8,5,6,1,9,4,10]
# release = [3,2,1,6,7,9,8,10]
dmax = max(duration)
dmin = min(duration)
rmax = max(release)
rmin = min(release)
wmax = max(weight)
wmin = min(weight)
sort_dur = sorted(duration)
sort_rel = sorted(release)
sum_m = 0
m = 0
for i in range(len(sort_dur)):
    sum_m += sort_dur[i]
    if sum_m >= rmax: # - min_rel is possible
        m = i+1
        break
print('The position m is: ', m)

The maximum possible duration is:  100
The maximum possible release date is:  2400
The maximum possible weight of completion time is:  1
The number of jobs is:  80
The SMS is represented as below: 
The real maximum duration is:  98
The real minimum duration is:  1
The real maximum release date is:  2316
The real minimum release date is:  4
The real maximum weight of completion time is:  1
The real minimum weight of completion time is:  1
The SMS problem is:  [(46, 2155, 1), (73, 418, 1), (97, 1372, 1), (81, 165, 1), (39, 714, 1), (48, 2294, 1), (37, 124, 1), (9, 1142, 1), (38, 1854, 1), (65, 2316, 1), (95, 1425, 1), (1, 1563, 1), (18, 693, 1), (71, 443, 1), (31, 896, 1), (80, 895, 1), (45, 1414, 1), (69, 2104, 1), (72, 224, 1), (30, 2115, 1), (10, 2281, 1), (88, 2010, 1), (4, 384, 1), (71, 1814, 1), (53, 897, 1), (9, 988, 1), (90, 738, 1), (2, 2123, 1), (94, 1617, 1), (29, 309, 1), (33, 2275, 1), (24, 1818, 1), (38, 559, 1), (30, 2138, 1), (22, 1612, 1), (28, 1990, 1), (55, 1603, 1), (

In [135]:
#############################################################
'''
Gurobi Model of Rank-based MILP model 
with integer variables w representing idle time between jobs
'''
m_r_w = Model("SMS_rank_w")

x_r_w = m_r_w.addVars(N_, N_, vtype=GRB.BINARY, name='x_r_w')
w_r_w = m_r_w.addVars(m, vtype=GRB.INTEGER, lb=0, name='w_r_w')

m_r_w.addConstrs((x_r_w.sum(i,'*') == 1 for i in range(N_)), name='one i')
m_r_w.addConstrs((x_r_w.sum('*',k) == 1 for k in range(N_)), name='one k')
m_r_w.addConstr((w_r_w[0] == quicksum(x_r_w[i,0]*release[i] for i in range(N_))), name='release 1')
m_r_w.addConstrs((w_r_w[k] + quicksum(w_r_w[j] + quicksum(x_r_w[i,j]*duration[i] for i in range(N_)) for j in range(k))
                 >= quicksum(x_r_w[r,k]*release[r] for r in range(N_)) for k in range(1,m)), name='release m')

obj1_r_w = quicksum((N_-k) * (w_r_w[k] + quicksum(x_r_w[i,k]*duration[i] for i in range(N_))) for k in range(m))
obj2_r_w = quicksum((N_-k) * quicksum(x_r_w[i,k]*duration[i] for i in range(N_)) for k in range(m,N_))
# obj3_r_w = quicksum(weight[j] * quicksum(w_r_w[k] + 
#            quicksum(x_r_w[i,k] * duration[i] for i in range(N_)) 
#            for k in range(j+1)) for j in range(m))
# obj4_r_w = quicksum(weight[j] * ( quicksum(w_r_w[k] + 
#            quicksum(x_r_w[i,k] * duration[i] for i in range(N_)) for k in range(m)) + 
#            quicksum(1 * quicksum(x_r_w[i,k] * duration[i] for i in range(N_)) 
#                     for k in range(m,j+1)) ) for j in range(m,N_))

m_r_w.setObjective(obj1_r_w+obj2_r_w)
m_r_w.modelSense = GRB.MINIMIZE
m_r_w.setParam("TimeLimit", 600)

m_r_w.update()
m_r_w.optimize()
runtime_r_w = m_r_w.Runtime
obj_val_r_w = m_r_w.objVal
print("The obj value is %i" % obj_val_r_w)
print("The run time is %f" % runtime_r_w)

# count = 0
# for z in m_r_w.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 226 rows, 6466 columns and 191891 nonzeros
Model fingerprint: 0xa57148c0
Variable types: 0 continuous, 6466 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 303437.00000
Presolve added 1 rows and 1 columns
Presolve time: 0.20s
Presolved: 227 rows, 6467 columns, 185266 nonzeros
Variable types: 0 continuous, 6467 integer (6401 binary)

Root relaxation: objective 1.193116e+05, 6085 iterations, 0.64 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 119311.629    0  207 303437.000 119311.629  60.7%     -    0s
H    0     0               

 26588 14693 121847.948   89  234 129695.000 121596.735  6.24%  34.5  477s
 26652 14719 121849.619   92  232 129695.000 121596.735  6.24%  34.7  480s
 26774 14807 121857.688   98  231 129695.000 121596.735  6.24%  35.1  486s
 26937 14883 121869.889  105  228 129695.000 121596.735  6.24%  35.5  493s
 27021 14946 121876.930  111  231 129695.000 121596.735  6.24%  35.7  497s
 27128 15009 121916.420  118  226 129695.000 121596.735  6.24%  35.9  500s
 27329 15088 122011.258  126  231 129695.000 121596.735  6.24%  36.3  508s
 27416 15148 122029.114  131  225 129695.000 121596.735  6.24%  36.6  513s
 27507 15203 122033.777  134  227 129695.000 121596.735  6.24%  36.9  517s
 27600 15252 122044.559  138  233 129695.000 121596.735  6.24%  37.2  521s
 27712 15307 122049.901  146  230 129695.000 121596.735  6.24%  37.6  525s
 27845 15348 122111.146  153  234 129695.000 121596.735  6.24%  37.9  530s
 28128 15499 122133.014  170  224 129695.000 121596.735  6.24%  38.5  539s
 28296 15587 122144.391  

In [136]:
#############################################################
'''
Gurobi Model of Rank-based MILP model 
with integer variables h representing start time of jobs
'''
m_r_h = Model("SMS_rank_h")

x_r_h = m_r_h.addVars(N_, N_, vtype=GRB.BINARY, name='x_r_h')
h_r_h = m_r_h.addVars(N_, vtype=GRB.INTEGER, lb=0, name='h_r_h')

m_r_h.addConstrs((x_r_h.sum(i,'*') == 1 for i in range(N_)), name='one i')
m_r_h.addConstrs((x_r_h.sum('*',k) == 1 for k in range(N_)), name='one k')
m_r_h.addConstrs((h_r_h[k] + quicksum(x_r_h[i,k]*duration[i] for i in range(N_)) 
                <= h_r_h[k+1] for k in range(m-1)), name='start before m-1')
m_r_h.addConstrs((h_r_h[k] + quicksum(x_r_h[i,k]*duration[i] for i in range(N_)) 
                == h_r_h[k+1] for k in range(m-1,N_-1)), name='start after m-1')
m_r_h.addConstrs((h_r_h[k] >= quicksum(x_r_h[i,k]*release[i] for i in range(N_)) for k in range(m)), name='release date before m')

obj1_r_h = quicksum(h_r_h[k] + quicksum(x_r_h[i,k]*duration[i] for i in range(N_)) for k in range(N_))
m_r_h.setObjective(obj1_r_h)
m_r_h.modelSense = GRB.MINIMIZE
m_r_h.setParam("TimeLimit", 600)

m_r_h.update()
m_r_h.optimize()

runtime_r_h = m_r_h.Runtime
obj_val_r_h = m_r_h.objVal
print("The obj value is %i" % obj_val_r_h)
print("The run time is %f" % runtime_r_h)

# count = 0
# for z in m_r_h.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 305 rows, 6480 columns and 24624 nonzeros
Model fingerprint: 0xe6a2810e
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14 rows and 14 columns
Presolve time: 0.03s
Presolved: 291 rows, 6466 columns, 23215 nonzeros
Variable types: 0 continuous, 6466 integer (6400 binary)
Found heuristic solution: objective 306150.00000

Root relaxation: objective 1.193116e+05, 7454 iterations, 0.37 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 119311.629    0  272 306150.000 119311.629  61.0%     -    0s
H    0     0             

 27749 14948 122880.607  133  361 128449.000 122146.411  4.91%  66.1  371s
 27913 15079 123574.023  139  353 128449.000 122146.411  4.91%  67.5  375s
 28267 15332     cutoff  146      128449.000 122146.411  4.91%  69.0  381s
H28516 14614                    128367.00000 122146.411  4.85%  69.3  384s
 28729 14684     cutoff  199      128367.000 122146.411  4.85%  69.2  387s
 29102 14750 128082.837  238  283 128367.000 122146.411  4.85%  69.2  391s
H29499 14013                    128225.00000 122146.424  4.74%  69.1  394s
 29694 14022 122178.484   35  425 128225.000 122146.581  4.74%  69.3  396s
 29952 14208 122401.922   54  401 128225.000 122146.581  4.74%  69.7  400s
 30566 14547 122533.136  122  376 128225.000 122146.581  4.74%  70.0  406s
 30877 14797 123745.345  163  345 128225.000 122146.581  4.74%  70.9  411s
 31306 15020 125075.438  191  331 128225.000 122146.581  4.74%  71.5  416s
 31709 15188 125363.369  209  303 128225.000 122178.686  4.72%  72.4  420s
 32140 15459 123145.929  

In [137]:
############################################################# Problematic -- Jason April 9 2020
'''
Gurobi Model of Linear-order MILP model 
'''
m_l = Model("SMS_line")

x_l = m_l.addVars(N_, N_, vtype=GRB.BINARY, name='x_l')
y_l = m_l.addVars(N_, vtype=GRB.INTEGER, lb=0, name='y_l')
big_M = N_*dmax

m_l.addConstrs((x_l[i,j] + x_l[j,i] == 1 
                for i in range(N_) for j in range(N_) if j>i), name='one order')
m_l.addConstrs((x_l[i,j] + x_l[j,k] + x_l[k,i] <= 2 
                for i in range(N_) for j in range(N_) for k in range(N_) 
                if i!=j and j!=k and i!=k), name='cycle elimination')
m_l.addConstrs((quicksum(duration[i]*x_l[i,j] for i in range(N_) if i!=j) + y_l[j] >= release[j] 
                for j in range(N_)), name='release date')
m_l.addConstrs((y_l[j] - y_l[i] >= big_M*(x_l[i,j]-1) 
                for i in range(N_) for j in range(N_) if j!=i), name='idle time')

obj1_l = quicksum(weight[j] * (duration[j] + y_l[j] + 
                  quicksum(duration[i]*x_l[i,j] for i in range(N_) if i!=j)) for j in range(N_))
m_l.setObjective(obj1_l)
m_l.modelSense = GRB.MINIMIZE
m_l.setParam("TimeLimit", 600)

m_l.update()
m_l.optimize()
runtime_l = m_l.Runtime
obj_val_l = m_l.objVal
print("The obj value is %i" % obj_val_l)
print("The run time is %f" % runtime_l)

# count = 0
# for z in m_l.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 502520 rows, 6480 columns and 1510560 nonzeros
Model fingerprint: 0xf58db73f
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+03]
Presolve removed 331800 rows and 3240 columns
Presolve time: 2.78s
Presolved: 170720 rows, 3240 columns, 518320 nonzeros
Variable types: 0 continuous, 3240 integer (3160 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7170    1.2053136e+05   0.000000e+00   6.319829e+09      5s
   20215    1.2130885e+05   0.000000e+00   1.095944e+07     10s
   29975    1.2131246e+05   0.000000e+00   1.813337e+07     15s
   41607    1.2134620e+05   0.000000e+00   1.183221e+07     20s
   55173    1.21

In [138]:
#############################################################
'''
Gurobi Model of Time-indexed MILP model 
'''
m_t = Model("SMS_time")

T = N_*dmax

x_t = m_t.addVars(N_, T, vtype=GRB.BINARY, name='x_t')

m_t.addConstrs((quicksum(t*x_t[i,t] for t in range(T)) >= release[i]
                for i in range(N_)), name='release constraints')
m_t.addConstrs((quicksum(x_t[i,t] for t in range(T)) == 1
                for i in range(N_)), name='one hot')
m_t.addConstrs((quicksum(quicksum(x_t[i,t2] for t2 in range(max(0,t-duration[i]+1),t+1)) 
                for i in range(N_)) <= 1 for t in range(T)), name='resource constraint')

obj1_t = quicksum(weight[i] * quicksum((t+duration[i])*x_t[i,t] for t in range(T)) for i in range(N_))
m_t.setObjective(obj1_t)
m_t.modelSense = GRB.MINIMIZE
m_t.setParam("TimeLimit", 600)

m_t.update()
m_t.optimize()
runtime_t = m_t.Runtime
obj_val_t = m_t.objVal
print("The obj value is %i" % obj_val_t)
print("The run time is %f" % runtime_t)

# count = 0
# for z in m_t.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 8000 rows, 627200 columns and 29184671 nonzeros
Model fingerprint: 0xede7915f
Variable types: 0 continuous, 627200 integer (627200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 357405.00000
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 30 rows and 97649 columns (presolve time = 10s) ...
Presolve removed 30 rows and 97649 columns (presolve time = 15s) ...
Presolve removed 30 rows and 97649 columns (presolve time = 20s) ...
Presolve removed 191 rows and 97762 columns (presolve time = 26s) ...
Presolve removed 202 rows and 97774 columns (presolve time = 35s) ...
Presolve removed 202 rows and 97774 columns (presolve time = 40s) ...
Pr

In [139]:
#############################################################
'''
Gurobi Model of Disjunctive MILP model 
'''
m_d = Model("SMS_disj")

z_d = m_d.addVars(N_, N_, vtype=GRB.BINARY, name='z_d')
x_d = m_d.addVars(N_, vtype=GRB.INTEGER, lb=release, name='x_d')
big_M = N_*dmax

m_d.addConstrs((x_d[i] >= x_d[j] + duration[j] - big_M*z_d[i,j] 
                for i in range(N_) for j in range(N_) if j>i), name='big_M 1')
m_d.addConstrs((x_d[j] >= x_d[i] + duration[i] - big_M*(1-z_d[i,j]) 
                for i in range(N_) for j in range(N_) if j>i), name='big_M 2')

obj1_d = quicksum(weight[i] * (x_d[i] + duration[i]) for i in range(N_))
m_d.setObjective(obj1_d)
m_d.modelSense = GRB.MINIMIZE
m_d.setParam("TimeLimit", 600)

m_d.update()
m_d.optimize()
runtime_d = m_d.Runtime
obj_val_d = m_d.objVal
print("The obj value is %i" % obj_val_d)
print("The run time is %f" % runtime_d)

# count = 0
# for z in m_d.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 6320 rows, 6480 columns and 18960 nonzeros
Model fingerprint: 0xf4c129db
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 8e+03]
Presolve removed 0 rows and 3240 columns
Presolve time: 0.03s
Presolved: 6320 rows, 3240 columns, 18960 nonzeros
Variable types: 0 continuous, 3240 integer (3160 binary)

Root relaxation: objective 1.012260e+05, 1523 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 101226.000    0  110          - 101226.000      -     -    0s
     0     0 102686.769    0  152          - 102686.769      -     -   

 37568 28521 105259.818   53  228 137865.000 104460.082  24.2%  15.9  241s
 37583 28531 121096.230  129  286 137865.000 104460.082  24.2%  15.9  245s
 37588 28534 122144.000  278  278 137865.000 104460.082  24.2%  15.9  252s
 37593 28538 105574.333   74  276 137865.000 104460.082  24.2%  15.9  256s
 37601 28543 126471.094  138  276 137865.000 104460.082  24.2%  15.9  261s
 37607 28547 113341.924  156  271 137865.000 104460.082  24.2%  15.9  267s
 37613 28551 104934.663   81  282 137865.000 104460.082  24.2%  15.9  271s
 37617 28554 111780.135  126  280 137865.000 104460.082  24.2%  15.9  275s
 37620 28556 118181.810  154  288 137865.000 104460.082  24.2%  15.9  280s
 37627 28560 105150.217   65  283 137865.000 104460.082  24.2%  15.9  286s
 37636 28566 130372.542  180  286 137865.000 104460.082  24.2%  15.8  290s
 37640 28569 110885.146  111  281 137865.000 104460.082  24.2%  15.8  295s
 37645 28572 108491.803   99  283 137865.000 104460.082  24.2%  15.8  300s
 37649 28575 114015.542  

In [140]:
#############################################################
'''
Gurobi Model of Disjunctive MILP model with redundant constraints
'''
m_d2 = Model("SMS_disj2")

z_d2 = m_d2.addVars(N_, N_, vtype=GRB.BINARY, name='z_d2')
x_d2 = m_d2.addVars(N_, vtype=GRB.INTEGER, lb=release, name='x_d2')
big_M = N_*dmax

m_d2.addConstrs((z_d2[i,j] + z_d2[j,i] == 1 
                for i in range(N_) for j in range(N_) if j>i), name='one order')
m_d2.addConstrs((z_d2[i,j] + z_d2[j,k] + z_d2[k,i] <= 2 
                for i in range(N_) for j in range(N_) for k in range(N_) 
                if i!=j and j!=k and i!=k), name='cycle elimination')
m_d2.addConstrs((x_d2[i] >= x_d2[j] + duration[j] - big_M*z_d2[i,j] 
                for i in range(N_) for j in range(N_) if j>i), name='big_M 1')
m_d2.addConstrs((x_d2[j] >= x_d2[i] + duration[i] - big_M*(1-z_d2[i,j]) 
                for i in range(N_) for j in range(N_) if j>i), name='big_M 2')

obj1_d2 = quicksum(weight[i] * (x_d2[i] + duration[i]) for i in range(N_))
m_d2.setObjective(obj1_d2)
m_d2.modelSense = GRB.MINIMIZE
m_d2.setParam("TimeLimit", 600)

m_d2.update()
m_d2.optimize()
runtime_d2 = m_d2.Runtime
obj_val_d2 = m_d2.objVal
print("The obj value is %i" % obj_val_d2)
print("The run time is %f" % runtime_d2)

# count = 0
# for z in m_d.getVars():
#     print(z)
#     print(z.x)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 502440 rows, 6480 columns and 1504160 nonzeros
Model fingerprint: 0x435770b9
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 8e+03]
Presolve removed 331800 rows and 3240 columns
Presolve time: 2.95s
Presolved: 170640 rows, 3240 columns, 511920 nonzeros
Variable types: 0 continuous, 3240 integer (3160 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    7402    1.0122664e+05   0.000000e+00   5.545778e+04      5s
   20759    1.0122600e+05   0.000000e+00   0.000000e+00      9s
   20759    1.0122600e+05   0.000000e+00   0.000000e+00      9s

Root relaxation: objective 1.012260e+05, 20759 iterations, 4.89 seconds
Total e

   618   662 106203.000   88  100 147067.000 103701.300  29.5%  36.9  145s
   760   809 106941.000  103   92 147067.000 103701.300  29.5%  32.0  150s
   920   984 108137.000  126   86 147067.000 103701.300  29.5%  27.8  156s
  1057  1145 109152.000  146   78 147067.000 103701.300  29.5%  25.3  161s
  1217  1292 109919.000  165   67 147067.000 103701.300  29.5%  23.0  166s
  1375  1460 111096.375  186   60 147067.000 103701.300  29.5%  21.2  171s
  1545  1627 112853.000  207   42 147067.000 103701.300  29.5%  19.6  177s
  1695  1702 114170.000  224   44 147067.000 103701.300  29.5%  18.4  188s
H 1696  1702                    146417.00000 103701.300  29.2%  18.4  188s
  1701  1790 113990.000  224   47 146417.000 103701.300  29.2%  18.4  190s
  1924  2041 116058.000  255   42 146417.000 103701.300  29.2%  16.9  198s
  2040  2149 120489.000  267   42 146417.000 103701.300  29.2%  16.4  200s
  2265  2402 122581.000  293   38 146417.000 103701.300  29.2%  15.5  206s
  2401  2540 124399.000  